In [6]:
from PIL import Image
import os

import sys

sys.path.append('../')
from cupagp.classifier import BinaryClassifier
import numpy as np

Load dataset and label set from file.

In [7]:
IH = 40
IW = 100

data = np.array([])
label = np.array([])

for root, ds, fs in os.walk('../datasets/uiuc/pos/'):
    num = 0
    for f in fs:
        image = Image.open('../datasets/uiuc/pos/' + f).convert('L')
        image = image.resize((IH, IW))
        image_arr = np.array(image).astype(float)
        # add to data and label
        data = np.append(data, image_arr)
        label = np.append(label, [1])

for root, ds, fs in os.walk('../datasets/uiuc/neg/'):
    num = 0
    for f in fs:
        image = Image.open('../datasets/uiuc/neg/' + f).convert('L')
        image = image.resize((IH, IW))
        image_arr = np.array(image).astype(float)
        # add to data and label
        data = np.append(data, image_arr)
        label = np.append(label, [0])

data = data.reshape(-1, IH, IW)
print(f'data shape: {data.shape}; label shape: {label.shape}')

data shape: (1050, 40, 100); label shape: (1050,)


Shuffle dataset and label.

In [8]:
# shuffle
data_label = list(zip(data, label))
np.random.shuffle(data_label)
data, label = zip(*data_label)
data, label = np.array(data), np.array(label)

Split train set and test set.

In [9]:
size = len(data)
train_size = size // 4 * 3
train_data, train_label, test_data, test_label = data[:train_size], label[:train_size], \
                                                 data[train_size:], label[train_size:]

size = len(test_data) // 2
valid_data, valid_label = test_data[:size], test_label[:size]
test_data, test_label = test_data[size:], test_label[size:]
print(f'train data shape: {train_data.shape}; valid data shape: {valid_data.shape}; test data shape: {test_data.shape}')

train data shape: (786, 40, 100); valid data shape: (132, 40, 100); test data shape: (132, 40, 100)


Create a binary classifier and send train_data, train_label, test_data, test_label to it.
This cell uses 'py_cuda' as cuda backend.
The arg 'eval_batch' is set to 20 to avoid memory allocate failed on the GPU.

In [10]:
import time

device = 'py_cuda'
eval_batch = 70

# if the two following args are set to None, the nvcc will automatically select one for compiling
# this might have mistakes, so you can specify these two args
# such as: cuda_arch = 'compute_75'; cuda_code = 'sm_75'
cuda_arch = None
cuda_code = None


classifier = BinaryClassifier(train_data, train_label, valid_data, valid_label, test_data, test_label,
                              eval_batch=eval_batch, device=device)

# train
classifier.train()

# test
classifier.run_test()

# remove the reference, to make sure the device side memory space is cleared
del classifier

[ Generation   ]  0
[ Best Fitness ]  0.7404580152671756
[ Best Program ]  [ Sub Sub G_Std LoG2 Lap Region_S(13, 0, 26, 26) Sub Sub G_Std Region_S(4, 39, 25, 25) G_Std Region_S(20, 26, 20, 20) Sub G_Std Region_S(15, 41, 21, 21) G_Std Region_R(10, 56, 30, 36) Sub G_Std Lap Gau11 Region_R(2, 27, 38, 46) G_Std Gau11 Gau1 Region_S(1, 73, 23, 23) ]

[ Generation   ]  1
[ Best Fitness ]  0.8269720101781171
[ Best Program ]  [ Sub Sub G_Std Sobel_X Region_S(12, 45, 26, 26) G_Std LBP GauXY Region_S(12, 32, 26, 26) Sub Sub Sub G_Std Region_S(16, 70, 21, 21) G_Std Region_S(14, 40, 21, 21) Sub G_Std Region_R(19, 74, 21, 21) G_Std Region_R(4, 33, 25, 62) G_Std Gau1 Region_R(17, 30, 23, 39) ]

[ Generation   ]  2
[ Best Fitness ]  0.8651399491094147
[ Best Program ]  [ Sub Sub G_Std Sobel_X Region_R(18, 48, 22, 25) G_Std LBP GauXY Region_S(12, 32, 26, 26) Sub Sub Sub G_Std Region_S(16, 70, 21, 21) G_Std Region_S(14, 40, 21, 21) Sub G_Std Region_R(19, 74, 21, 21) G_Std Region_R(4, 33, 25, 62) G_Std 